In [43]:
import pandas as pd
scenario1 = pd.read_csv("./editdata/scenario1_clicklogs.csv")
scenario2 = pd.read_csv("./editdata/scenario2_clicklogs.csv")


In [44]:
import pandas as pd

# timestamp 형식 변환
scenario1['timestamp'] = pd.to_datetime(scenario1['timestamp'])

# 사용자별로 정렬
scenario1 = scenario1.sort_values(by=['username', 'timestamp'])

# 시간 차이 계산
scenario1['duration_sec'] = scenario1.groupby('username')['timestamp'].diff().dt.total_seconds()

# 사용자별 시나리오 블록 나누기: "StartButton - scenario 1" 기준으로 시나리오 시작점 표시
scenario1['is_start'] = (scenario1['button'] == 'StartButton - scenario 1').astype(int)

# 사용자 내에서 누적합으로 각 시나리오 그룹 구분 (여기선 내부적으로만 사용, 컬럼 생성 X)
scenario1['temp_group'] = scenario1.groupby('username')['is_start'].cumsum()

# 각 시나리오 그룹별로 duration 합산
total_duration_per_group = (
    scenario1
    .fillna({'duration_sec': 0})
    .groupby(['username', 'temp_group'])['duration_sec']
    .sum()
    .reset_index()
    .rename(columns={'duration_sec': 'total_duration_sec'})
)

# 원래 데이터프레임에 merge (temp_group 기준)
scenario1 = scenario1.merge(total_duration_per_group, on=['username', 'temp_group'], how='left')
scenario1['duration_sec'] = scenario1['duration_sec'].fillna(0)
# 불필요한 중간 컬럼 제거
scenario1.drop(columns=['is_start', 'temp_group'], inplace=True)
scenario1['scenario'] = 'scenario 1'

scenario1.to_csv("scenario1_added_duration.csv")

In [45]:
import pandas as pd

# timestamp 형식 변환
scenario2['timestamp'] = pd.to_datetime(scenario2['timestamp'])

# 사용자별로 정렬
scenario2 = scenario2.sort_values(by=['username', 'timestamp'])

# 시간 차이 계산
scenario2['duration_sec'] = scenario2.groupby('username')['timestamp'].diff().dt.total_seconds()

# 사용자별 시나리오 블록 나누기: "StartButton - scenario 1" 기준으로 시나리오 시작점 표시
scenario2['is_start'] = (scenario2['button'] == 'StartButton - scenario 2').astype(int)

# 사용자 내에서 누적합으로 각 시나리오 그룹 구분 (여기선 내부적으로만 사용, 컬럼 생성 X)
scenario2['temp_group'] = scenario2.groupby('username')['is_start'].cumsum()

# 각 시나리오 그룹별로 duration 합산
total_duration_per_group = (
    scenario2
    .fillna({'duration_sec': 0})
    .groupby(['username', 'temp_group'])['duration_sec']
    .sum()
    .reset_index()
    .rename(columns={'duration_sec': 'total_duration_sec'})
)

# 원래 데이터프레임에 merge (temp_group 기준)
scenario2 = scenario2.merge(total_duration_per_group, on=['username', 'temp_group'], how='left')
scenario2['duration_sec'] = scenario2['duration_sec'].fillna(0)
# 불필요한 중간 컬럼 제거
scenario2.drop(columns=['is_start', 'temp_group'], inplace=True)
scenario2['scenario'] = 'scenario 2'
scenario2.to_csv("scenario2_added_duration.csv")